Цей код створює 2 окремі датафрейми:
- df_duration_ouliers - містить аутлаєри по часу поїздки > 24 годин.
- df - містить сирі дані очищені від аутлаєрів > 24 годин.

В обох датафреймах міститься ще 2 додаткові колонки: started_at_dt і ended_at_dt, що містять час початку і кінця поїздки в типі datetime в таймзомі міста Осло.

In [1]:
import pandas as pd
import datetime
import time
import helpful_functions
import pytz

In [2]:
city_name = 'oslo'        # можливі варіанти: olso, bergen, trondheim

start_month = 4           # 1 - 12
start_year = 2019         # 2019 - 2022

end_month = 6             # 1 - 12
end_year = 2022           # 2019 - 2022

data_folder = './data'

In [3]:
dates_str_list = helpful_functions.generate_months_in_period(
    start_month=start_month, 
    start_year=start_year, 
    end_month=end_month, 
    end_year=end_year, 
    sep="_")

In [4]:
start = time.time()
df = pd.read_csv(f"{data_folder}/{city_name}_{dates_str_list[0]}.csv")
for date in dates_str_list[1:]:
    df = pd.concat([df, pd.read_csv(f"{data_folder}/{city_name}_{date}.csv")])
end = time.time()
print("Зчитування: ", end - start, "с. \n")
print("Розмір початкової таблиці: ", df.shape)

Зчитування:  44.2213351726532 с. 

Розмір початкової таблиці:  (5833647, 13)


In [5]:
strptime_pattern = "%Y-%m-%d %H:%M:%S%z"
timezone = pytz.timezone('Europe/Oslo')

start = time.time()
df['started_at_dt'] = df['started_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
df['ended_at_dt'] = df['ended_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
end = time.time()
print("Трансформування: ", end - start, "с.")
df.head()

Трансформування:  217.26721382141113 с.


,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude,started_at_dt,ended_at_dt
0,2019-04-02 22:18:47.926000+00:00,2019-04-02 22:24:25.427000+00:00,337,401,Schous plass,nærmest Thorvald Meyers gate,59.920259,10.760629,381,Grønlands torg,ved Tøyenbekken,59.91252,10.76224,2019-04-03 00:18:47+02:00,2019-04-03 00:24:25+02:00
1,2019-04-02 22:25:54.203000+00:00,2019-04-02 22:27:36.114000+00:00,101,556,Pilestredet,ved Sporveisgata,59.926224,10.729625,556,Pilestredet,ved Sporveisgata,59.926224,10.729625,2019-04-03 00:25:54+02:00,2019-04-03 00:27:36+02:00
2,2019-04-02 22:26:39.308000+00:00,2019-04-02 22:31:59.445000+00:00,320,561,Kirkeveien,nord for Gørbitz' gate,59.933558,10.726426,471,Marienlyst,i Suhms gate,59.932454,10.721769,2019-04-03 00:26:39+02:00,2019-04-03 00:31:59+02:00
3,2019-04-03 03:09:58.992000+00:00,2019-04-03 03:12:00.589000+00:00,121,393,Hans Nielsen Hauges plass,Mellom Åsengata og Nordkappgata,59.939244,10.774319,393,Hans Nielsen Hauges plass,Mellom Åsengata og Nordkappgata,59.939244,10.774319,2019-04-03 05:09:58+02:00,2019-04-03 05:12:00+02:00
4,2019-04-03 03:10:02.415000+00:00,2019-04-03 03:14:44.537000+00:00,282,597,Fredensborg,ved rundkjøringen,59.920995,10.750358,557,Akersgata,ved Grensen,59.914197,10.742883,2019-04-03 05:10:02+02:00,2019-04-03 05:14:44+02:00


Створюємо 2 датафрейми: 
- df_duration_ouliers - містить аутлаєри по часу поїздки > 24 годин. Потрібний для подальшого аналізу цих аутлаєрів.
- df - містить сирі дані очищені від аутлаєрів > 24 годин. Для подальшого аналізу.

In [6]:
df_duration_ouliers = df[df['duration'] > 24 * 60 * 60]
df = df.drop(df[df['duration'] > 24 * 60 * 60].index)

In [18]:
# df.to_csv('1.csv')